In [1]:
#ML SS Single Instance: verification that an instance yields the equilibrium suggested by theory as well as by computation
%load_ext autoreload
%autoreload 2
from utils.imports import *
from utils.single_stage import *

In [2]:
dist  = 'uniform'
npts = 100
theta = 0.5
ca,cb,la,lb,sa,sb = ml_get_example_in_region(3,dist)
maxpx = ca+5

{'ca': 1, 'cb': 0, 'la': 2, 'lb': 2}
{'paa_sst': 1.98, 'pba_sst': 0.99, 'pab_sst': 1.32, 'pbb_sst': 1.65}


In [3]:
paa_arr,pba_arr,objaa,objba,constraint_a_arr = ml_get_payoff_matrices_state_a(ca,cb,maxpx,npts,dist,la)
pab_arr,pbb_arr,objab,objbb,constraint_b_arr = ml_get_payoff_matrices_state_b(ca,cb,maxpx,npts,dist,lb)

In [4]:
compute_single_stage_equilibrium(objaa,objba,objbb,objab,paa_arr,pba_arr,pbb_arr,pab_arr)

{'paa': 2.01,
 'pba': 1.03,
 'pbb': 1.697,
 'pab': 1.354,
 'objaa': 0.515,
 'objba': 0.505,
 'objbb': 1.406,
 'objab': 0.061}

In [5]:
df = ml_get_metric_arrs_vs_camcb(np.array([ca]),cb,la,lb,dist,theta,flag_theory=True)
df

,paa,pba,pbb,pab,objaa,objba,objbb,objab,marketshare_a,marketshare_b,total_profit_a,total_profit_b,prob_purchase_a_from_a,prob_purchase_b_from_b
0,1.98,0.99,1.65,1.32,0.4949,0.49005,1.37775,0.0528,0.335,0.665,0.27385,0.9339,0.505,0.835
